# Bootstraping procedure in pricing Credit Default swaps
## Introduction
Credit Default Swaps (CDS) are financial instruments used to transfer credit risk from one party to another. They function as insurance contracts on the creditworthiness of a reference entity, such as a corporation or a sovereign nation.

To accurately price a CDS, the key objective is to determine the implied default probabilities of the reference entity over various maturities, which is where the bootstrapping procedure comes into play. Bootstrapping is a method used to extract these implied survival probabilities from a set of market-quoted CDS spreads with different maturities. 
## Credit Default Swaps
A CDS is a financial derivative that allows an investor to swap or offset their credit risk with that of another
investor. The lender buys a CDS from another investor who agrees to reimburse the first if the borrower defaults.
CDS are maintained via an ongoing premium payment similar to an insurance.

*Example : Suppose a company issues a $100 bond of maturity 10 years with regular interest payments
throughout the bond’s life. Because the debt issuer cannot guarantee his ability to pay premiums the debt
buyer can purchase a CDS to transfer this risk to another investor, who agrees to pay them in the event the
debt issuer defaults on its obligation*.

Since the most of debts tend to have long maturities the buyer of the CDS pays a the seller until the contract’s
maturity. In return the CDS seller agrees that it will pay the CDS buyer the security’s value as well as all
interest payments that would have been paid between that time and the maturity date if their is a credit
event with a recovery rate.

The CDS are triggered with what we call a credit event which are agreed upon when the CDS is purchased
and are mentioned in the contract.\
The CDS can be used for multiple reasons : 
1. Speculating.
2. Hedging.
3. Arbitrage.

## Bootstrape 
In finance, bootstrapping is a method used to construct a complete yield curve or credit curve from a set of market data points, such as bond prices, interest rates, or CDS spreads. The core idea is to iteratively derive zero-coupon rates, discount factors, or survival probabilities from quoted data to build a curve that spans a range of maturities.

It’s named “bootstrapping” because each new point on the curve is built sequentially, using information from previously derived points—similar to "pulling oneself up by one's bootstraps."
## Bootstraping in Pricing Credit default swaps
In the context of CDS pricing, bootstrapping is used to derive the credit curve or the survival probability curve from CDS spreads. Here’s how the process works:
1. Step 1: Gather Market Data: Collect the CDS spreads for different maturities (e.g., 1-year, 3-year, 5-year CDS spreads).
2. Step 2: Compute the Premium Leg and Default Leg: For each CDS maturity, calculate the present value of the premium leg (the periodic payments made by the CDS buyer) and the default leg (the expected payment made by the CDS seller if a credit event occurs).
3. Step 3: Solve for the Survival Probability: Use an iterative procedure to determine the implied survival probabilities that equate the present value of the premium leg to the present value of the default leg. This is done by solving for the hazard rate ℎ that satisfies the equation:
$$\text{Premium Leg} = \text{Default leg }$$
4. Step 4: Build the Survival Probability Curve: Repeat the procedure for all maturities to construct the entire survival probability curve. The curve captures the probability that the reference entity will not default by each point in time.





In our context we will model a CDS as a class, and then we will calculate it's default leg and premium leg, using the survival probability and discount factor.
## The CDS Class
### Calculating the discount factor 
The `getDiscountFactor` function is designed to calculate the discount factor corresponding to a given time `t`  using a yield curve. 

To calculate the discount factor we will use the following variables as outputs : 
- `yieldcurveTenor` : A list of time points (maturities) for which yield rates are provided.
- `yieldcurveRate`: A list of corresponding interest rates (or zero-coupon rates) for each maturity in           yieldcurveTenor.
We know that the discount facotr is : $Discount = e^{-tr(t)}$ where $r(t)$ is the zero coupoun rate at time t which are stored in the `yieldcurveRate` variable. 
For times between two maturities [$T_i, T_{i+1}$] we will use a simple linear interpolation as follows : 
$$yieldcurveRate = yieldcurveRate_i + \frac{yieldcurveRate_{i+1}-yieldcurveRate_i}{yieldcurveTenor_{i+1} - yieldcurveTenor_i}\times (t-yieldcurveTenor_i)$$

In [3]:
import numpy as np
def getDiscountFactor(yieldcurveTenor, yieldcurveRate, t):
        
        result = -1 #The default value indicating an error
        min_time_index = 0
        max_time_index = len(yieldcurveTenor) - 1

        if t < 0:
            result = - 1 #returns an error
        elif t == 0:
            result = 1.0 # we are situated in the present here so no discounting
        elif t > 0 and t <= yieldcurveTenor[min_time_index]:
            result = np.exp(-t*yieldcurveRate[0])#When the t is inferieur we use the first rate
        elif t >= yieldcurveTenor[max_time_index]:
            result = np.exp(-t*yieldcurveRate[-1]) #When the t is superieure we use the last rate
        else:
            for i in range(max_time_index):
                if t >= yieldcurveTenor[i] and t < yieldcurveTenor[i+1]:
                    yield_interpolated = yieldcurveRate[i] + (yieldcurveRate[i+1] - yieldcurveRate[i]) / \
                        (yieldcurveTenor[i+1]-yieldcurveTenor[i]
                         ) * (t-yieldcurveTenor[i])
                    result = np.exp(-t*yield_interpolated)
        return result

### Calculating the survival probabilities
The function `getSurvivalProbability` calculates the survival probability of a reference entity up to a given time `t`, using a given credit curve. This probability is a crucial component in pricing credit default swaps (CDS) as it helps determine the likelihood that the entity will not default by a specific time.
 
To calculate the survival probability we will use the following varibales as outputs : 
- `creditcurveTenor` :  Contains the time points (tenors) at which the survival probabilities are defined.
- `creditcurveSP` : The Survival probabilities corresponding to the tenors in creditcurveTenor.

Internally one important variable that we will be using is the hazard rate, if we denote $\tau$ as the time fo default, then $h$ is the **instantaneous default intensity or failure rate**, defined as : 
$$ h = \lim_{\Delta t \rightarrow 0} \frac{\mathbb{P}(t \leq \tau < t + \Delta t)}{\Delta t}$$
Moreover, if we denote the survival prbability as $P(t) = \mathbb{P}{\tau >t}$ then $h = \frac{f(t)}{P(t)}$ or in other terms $h(t) = -\frac{d}{dt} \ln(P(t)) = -\frac{1}{P(t)}\frac{dP(t)}{dt}$.
Solving for this equation will gives us : 
$$P(t) = \exp\left(-\int_{0}^{t} h(u)du\right)$$
In teh special case of a constant hazard rate wiche will be the assumption taken in our case, then this formula soimples out to : 
$$h = -\frac{P(T)}{T}$$
To get the survival probability at time $t$ we can easily do it by : 
$$P(t) = \exp\left(-\frac{\ln(P(T))}{T} \times t \right)$$

As the case for the discount factor for times between two tenors [$T_i, T_{i+1}$], we interploate the survival probability by  : 
$$h = - \frac{\left(\frac{P_{i+1}}{P_i}\right)}{T_{i+1}-T_i}$$

In [4]:
def getSurvivalProbability(creditcurveTenor, creditcurveSP, t):
    result = -1 #the default error value
    min_time_index = 0
    max_time_index = len(creditcurveTenor) - 1
    if t < 0:
        result = -1
    elif t == 0:
        result = 1 # at the present the probability of not defaulting is 1 since no default has occured yet
    elif t > 0 and t <= creditcurveTenor[min_time_index]:
        h = -np.log(creditcurveSP[0] / creditcurveTenor[min_time_index]) #When t is small we take the first SP
        result = np.exp(-h*t)
    elif t == creditcurveTenor[max_time_index]:
        result = creditcurveSP[-1]
    elif t > creditcurveTenor[max_time_index]:
        h = 0
        if len(creditcurveTenor) == 1: # When we only have on tenor then we take it ti calculate the h
            h = h = - np.log(creditcurveSP[-1]) / \
                creditcurveTenor[max_time_index]
        else: #In the case of more we take the average of two last tenors
            #We take the average to reflects the decline in the survival probability 
            #And provides a trend that can be reasonably extended to times beyond the last tenor smoothly.
            h = - np.log(creditcurveSP[-1]/creditcurveSP[-2]) / \
                (creditcurveTenor[-1]-creditcurveTenor[-2])
            result = creditcurveSP[-1] * \
                np.exp(-(t - creditcurveTenor[max_time_index])*h)
    else:  # where t is in between min_time and max_time
        for i in range(max_time_index):
            if t >= creditcurveTenor[i] and t < creditcurveTenor[i+1]:
                h = -np.log(creditcurveSP[i+1]/creditcurveSP[i]) / \
                    (creditcurveTenor[i+1]-creditcurveTenor[i])
                result = creditcurveSP[i] * \
                    np.exp(-(t-creditcurveTenor[i])*h)
                
    return result


### Calculating the Premium Leg 
The `calculatePremiumLeg` function represents the stream of periodic premium payments made by the CDS buyer to the seller, in exchange for protection against the default of a reference entity. The function considers both the regular premium payments and the accrued premium in the event of default.

To calculate the premium leg we will use the follwing variables as outputs : 
- `reditcurveTenor` 
- `creditcurveSP` 
- `yieldcurveTenor`  
- `yieldcuvreRate` 
- `cdsMaturity` : The maturity of the CDS contract in years.
- `num_premium_year` : The number of premium payments made per year (e.g., 4 for quarterly payments).
- `accruedPremiumFlag` : A boolean flag indicating whether to consider accrued premium in the case of default.
- `spread` : The CDS spread (in percentage points) paid by the buyer of the CDS to the seller.
- `h` : The hazard rate used for extrapolation of survival probabilities beyond the known tenors.

The premium leg is : 
$$PL (t)=\sum_{t_m\leq t} Spread\times (t_m-t_{m-1})\mathbb{E}(B(t,t_m)1_{\tau>t_m}) =  Spread \times \left( \sum_{n=1}^{N} dt \times \text{DF}(t_n) \times S(t_n) + \text{Accrued Premium} \right)
$$
In the case where the CDS maturity is whitin the Credit curve Tenor then we have all the necessary information to calculate the premium leg.

However, in the case where the cds maturity is bigger than the biggest credit Tenor, then it will calculate the annuity and accrued premium similarly for payment dates up to the last available credit curve tenor (`creditcurveTenor[-1]`).\
For payment dates beyond the maximum tenor, the function extrapolates the survival probabilities using the last known hazard rate 
$h$ with the formula, here h is the estimated from the last two Tenors (except when we only have one Tenor) : 
$$S(t_n) = S(T_{Last})\exp(-h(t_n-T_{Last}))$$
As for payment dates between the maximum tenor and CDS maturity, the survival probability is calculated as:
$$S(t_{n-1}) = S(T_{Last})\exp(-h(t_{n-1}-T_{Last}))$$


In [5]:
def calculatePremiumLeg( creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate, cdsMaturity,
                            num_premium_year, accruedPremiumFlag, spread, h):
        max_time_index = len(creditcurveTenor) - 1
        if max_time_index > 0 and cdsMaturity <= creditcurveTenor[max_time_index]:
            annuity = 0
            accruedPremium = 0
            N = int(cdsMaturity*num_premium_year) #Total number of premium payments
            for n in range(1, N+1):
                tn = n / num_premium_year
                tnm1 = (n-1) / num_premium_year
                dt = 1.0 / num_premium_year
                annuity += dt * \
                    getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn) * \
                    getSurvivalProbability(
                        creditcurveTenor, creditcurveSP, tn)
                if accruedPremiumFlag:
                    accruedPremium += 0.5*dt*self.getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*(
                        self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1) - self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tn))
            
            return spread*(annuity+accruedPremium)
        else:  # When the cds maturity is beyond our current credit curve, we need to estimate the survival probability for payment beyond credit curve
            annuity = 0
            accruedPremium = 0
            N = int(cdsMaturity*num_premium_year)
            M = creditcurveTenor[max_time_index] * \
                num_premium_year if max_time_index > 0 else 0 #Max payment period covered the credit curve

            for n in range(1, N+1):
                if n <= M:
                    tn = n/num_premium_year
                    tnm1 = (n-1)/num_premium_year
                    dt = 1.0 / num_premium_year

                    annuity += dt * self.getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn) * \
                        self.getSurvivalProbability(
                            creditcurveTenor, creditcurveSP, tn)
                    if(accruedPremiumFlag):
                        accruedPremium += 0.5*dt*self.getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*(
                            self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1) -
                            self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tn))
                else:
                    tn = n/num_premium_year
                    tnm1 = (n-1)/num_premium_year
                    tM = M / num_premium_year
                    dt = 1.0 / num_premium_year

                    survivalProbability_n = self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tM) * \
                        np.exp(-h*(tn - tM))
                    survivalProbability_nm1 = 0
                    if tnm1 <= tM:
                        survivalProbability_nm1 = self.getSurvivalProbability(
                            creditcurveTenor, creditcurveSP, tnm1)
                    else:
                        survivalProbability_nm1 = self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tM) *\
                            np.exp(-h*(tnm1 - tM))

                    annuity += dt * self.getDiscountFactor(
                        yieldcurveTenor, yieldcurveRate, tn)*survivalProbability_n
                    if accruedPremiumFlag:
                        accruedPremium += 0.5*dt * self.getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn) *\
                            (survivalProbability_nm1-survivalProbability_n)
            return spread*(annuity+accruedPremium)


### Calculate the Default Leg
The `calculateDefaultLeg` function will calculate the default leg of a credit default swap (CDS). This leg represents the expected present
value of cash flows that would be paid in the event of a default by the reference entity.

To calculate the default leg, we will use the following variables as outputs : 
- `creditcurveSP`
- `creditcurveTenor`
- `yiledcurveTenor`
- `yieldcurveRate`
- `cdsMaturity`
- `h`
- `num_default_rate` : A parameter indicating the number of premium payment periods per year (e.g., monthly, quarterly, annually). This is used to calculate the timing of cash flows. 
- `recoveryRate` : The proportion of the exposure that is expected to be recovered in the event of a default. A value of 0.4 would imply that 40% of the exposure is recoverable.

The default leg is calculated as 
 $$\text{Default Leg} = (1 - \text{recoveryRate}) \times \left( 
\sum_{n=1}^{N} \text{DF}(t_n) \times \left( S(t_{n-1}) - S(t_n) \right) 
\right)$$

For the case where the cds maturity is whitin the credit curve Tenor, then we can calculate the default leg since we have all the needed information.
However, when the maturity is beyond the credit curve, then we calculate the annuity for the period between the start and the last known credit curve Tenor, as for the remaining time we have  the probability of surviving until time $t_n$ after $T_{Last}$: 
$$S(t_n) = S(T_{Last})e^{-h(t_n-T_{Last})}$$
As for the previous periode if it belongs to the credit curve , then we proceed normally, but if it's bigger then : 
$$S(t_{n-1}) = S(T_{Last})e^{-h(t_{n-1}-T_{Last})}$$



In [6]:
def calculateDefaultLeg(self, creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate,
                            cdsMaturity, num_default_year, recoveryRate, h):
        spread = self.spread
        max_time_index = len(creditcurveTenor) - 1
        accruedPremiumFlag = self.accruedPremium  # True or False
        if max_time_index > 0 and cdsMaturity <= creditcurveTenor[max_time_index]:
            annuity = 0
            N = int(cdsMaturity * num_default_year)
            for n in range(1, N+1):
                tn = n / num_default_year
                tnm1 = (n-1) / num_default_year
                annuity += self.getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*(
                    self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1) -
                    self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tn))
            return (1-recoveryRate)*annuity
        else:  # cdsMaturity > creditcurveTenor[max_time_index]
            annuity = 0
            N = int(cdsMaturity*num_default_year)
            M = creditcurveTenor[max_time_index] * \
                num_default_year if max_time_index > 0 else 0

            for n in range(1, N+1):
                if n <= M:
                    tn = n / num_default_year
                    tnm1 = (n-1) / num_default_year
                    annuity += self.getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn)*(
                        self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tnm1) -
                        self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tn))
                else:  # n > m
                    tM = M / num_default_year
                    tn = n / num_default_year
                    tnm1 = (n-1) / num_default_year

                    survivalProbability_n = self.getSurvivalProbability(creditcurveTenor, creditcurveSP, tM) *\
                        np.exp(-h*(tn-tM))
                    if tnm1 <= tM:
                        survivalProbability_nm1 = self.getSurvivalProbability(
                            creditcurveTenor, creditcurveSP, tnm1)
                    else:
                        survivalProbability_nm1 = self.getSurvivalProbability(
                            creditcurveTenor, creditcurveSP,  tM) * np.exp(-h*(tnm1 - tM))
                    annuity += self.getDiscountFactor(yieldcurveTenor, yieldcurveRate, tn) * (
                        survivalProbability_nm1 - survivalProbability_n)
                    

            return (1-recoveryRate)*annuity

## The Bootstraping procedure 
As mentionned before the Goal here is find the hazard rate that will equate for the default and premium Leg of market quoted CDS. 

For this purpose, the first function will be to calculate the difference between the premium and default leg of a CDS contract given a hazard rate $h$.

In [7]:
def objfunFindHazardRate(h):
    # print(cdsMaturity)
    premLeg = calculatePremiumLeg(creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate, cdsMaturity, premiumFrequency, 
                                 accruedPremium, spread,h)
    defaultLeg = calculateDefaultLeg(creditcurveTenor, creditcurveSP, yieldcurveTenor, yieldcurveRate, cdsMaturity, defaultFrequency, 
                                 recoveryRate, h)
    return premLeg - defaultLeg

Now that we have this helper function, we programm the `bootstrapCDSspread` to find for each makret quoted CDS the hazard rate that will equate between the premium and default leg, this hazard rate will be crucial in calculating the survival probability.

We will use the Newton methode to solve for h the equation $premiumLeg(h) = defaultLeg(h)$

In [8]:
def bootstrapCDSspread(yieldcurveTenor, yieldcurveRate, cdsTenors, cdsSpreads, premiumFrequency, defaultFrequency, 
                       accruedPremium, recoveryRate):
    yieldcurveLength = len(yieldcurveTenor)
    cdsTenorsLength = len(cdsTenors)
    
    newcreditcurveLength = 0
    newcreditcurve = []
    survprob = [None]*cdsTenorsLength
    hazardRate = [None]*cdsTenorsLength
    global creditcurveSP 
    creditcurveSP = []
    global creditcurveTenor 
    creditcurveTenor = []
    for i in range(cdsTenorsLength):
        global cdsMaturity 
        cdsMaturity = cdsTenors[i]
        global spread
        spread = cdsSpreads[i]
        # print(cdsMaturity, spread)
        h = newton(objfunFindHazardRate, 0.01)
        hazardRate[i] = h
        if i==0:
            survprob[i] = np.exp(-hazardRate[i]*cdsTenors[i])
        else:
            survprob[i] = survprob[i-1]*np.exp(-hazardRate[i]*(cdsTenors[i]-cdsTenors[i-1]))
        creditcurveTenor.append(cdsTenors[i])
        creditcurveSP.append(survprob[i])
    return hazardRate, survprob